# 前処理

[参考](https://qiita.com/Hironsan/items/2466fe0f344115aff177)

## データロード (nucc)

In [104]:
import glob
def loadNucc():
    data = []
    for file in glob.glob('./nucc/*'):
        d = open(file)
        data.append(d.read())
        d.close()
        # break
    return data
docData = loadNucc()

## クリーニング

In [105]:
import re
def cleanDocNucc(docData):
    for idx,doc in enumerate(docData):
        # ヘッダー & 話者プレフィックス削除
        tmp = re.sub(re.compile(r'^＠.*|^[FM](\d{3})：|^Ｘ：', re.MULTILINE),"",doc)
        # （おー）/＜笑い＞など削除
        tmp = re.sub(re.compile(r'（[^（]+）|＜[^＜]+＞', re.MULTILINE),"",tmp)
        # ＊置換
        tmp = re.sub(re.compile(r'＊', re.MULTILINE),"",tmp)
        # 改行を削除
        tmp = re.sub(re.compile(r'\n', re.MULTILINE)," ",tmp)
        docData[idx] = tmp
    return docData

docData = cleanDocNucc(docData)

## 単語に分割(janome編)

In [106]:
from janome.tokenizer import Tokenizer
def toWakati(docData):
    result = []
    tokenizer = Tokenizer()
    for doc in docData:
        tokens = tokenizer.tokenize(doc)
        r = []
        for token in tokens:
            if(token.part_of_speech.split(',')[0] != "記号"):
                r.append(token.surface)
        if(0<len(r)):
            result.append(r)
    return result
docData = toWakati(docData)

## 単語の正規化

In [107]:
def normalize(docData):
    #文字種の統一 ？
    #数字の置き換え　？
    #単語の統一　（ソニー > sony　みたいな）
    return docData

docData = normalize(docData)

## 学習

In [119]:
from gensim import models
from gensim.models.doc2vec import LabeledSentence

def word2vec(docData):
    sentences = []
    for idx,doc in enumerate(docData):
        sentences.append(LabeledSentence(words=doc, tags=[idx]))
    model = models.Doc2Vec(sentences, dm=0, size=300, window=15, alpha=.025,
        min_alpha=.025, min_count=1, sample=1e-6)
    for epoch in range(200):
        print('Epoch: {}'.format(epoch + 1))
        model.train(sentences,total_examples=model.corpus_count, epochs=model.iter)
        model.alpha -= (0.025 - 0.0001) / 19
        model.min_alpha = model.alpha
        
    model.save("word2vec.model")
word2vec(docData)